In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [3]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [4]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [5]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [6]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 73817.3594 Validation Accuracy: 0.156250
Epoch  1, Batch   2 -Loss: 54045.3984 Validation Accuracy: 0.191406
Epoch  1, Batch   3 -Loss: 42674.7617 Validation Accuracy: 0.199219
Epoch  1, Batch   4 -Loss: 42375.2109 Validation Accuracy: 0.207031
Epoch  1, Batch   5 -Loss: 33329.6484 Validation Accuracy: 0.218750
Epoch  1, Batch   6 -Loss: 29403.0840 Validation Accuracy: 0.253906
Epoch  1, Batch   7 -Loss: 26382.5645 Validation Accuracy: 0.273438
Epoch  1, Batch   8 -Loss: 25808.5586 Validation Accuracy: 0.269531
Epoch  1, Batch   9 -Loss: 24679.2578 Validation Accuracy: 0.242188
Epoch  1, Batch  10 -Loss: 17698.8613 Validation Accuracy: 0.257812
Epoch  1, Batch  11 -Loss: 19176.1836 Validation Accuracy: 0.273438
Epoch  1, Batch  12 -Loss: 18088.5488 Validation Accuracy: 0.265625
Epoch  1, Batch  13 -Loss: 19324.4297 Validation Accuracy: 0.281250
Epoch  1, Batch  14 -Loss: 14059.6680 Validation Accuracy: 0.308594
Epoch  1, Batch  15 -Loss: 17429.7148 Validation

Epoch  1, Batch 122 -Loss:  3405.1528 Validation Accuracy: 0.710938
Epoch  1, Batch 123 -Loss:  3189.1763 Validation Accuracy: 0.703125
Epoch  1, Batch 124 -Loss:  3912.8164 Validation Accuracy: 0.707031
Epoch  1, Batch 125 -Loss:  3849.0903 Validation Accuracy: 0.699219
Epoch  1, Batch 126 -Loss:  3916.6230 Validation Accuracy: 0.695312
Epoch  1, Batch 127 -Loss:  3088.2578 Validation Accuracy: 0.703125
Epoch  1, Batch 128 -Loss:  3771.5874 Validation Accuracy: 0.703125
Epoch  1, Batch 129 -Loss:  3933.9084 Validation Accuracy: 0.699219
Epoch  1, Batch 130 -Loss:  3270.4019 Validation Accuracy: 0.703125
Epoch  1, Batch 131 -Loss:  3176.2559 Validation Accuracy: 0.699219
Epoch  1, Batch 132 -Loss:  2474.8628 Validation Accuracy: 0.730469
Epoch  1, Batch 133 -Loss:  2153.8164 Validation Accuracy: 0.730469
Epoch  1, Batch 134 -Loss:  4538.4590 Validation Accuracy: 0.726562
Epoch  1, Batch 135 -Loss:  3859.1567 Validation Accuracy: 0.722656
Epoch  1, Batch 136 -Loss:  2819.3999 Validation

Epoch  1, Batch 243 -Loss:  2158.1089 Validation Accuracy: 0.781250
Epoch  1, Batch 244 -Loss:  2417.0244 Validation Accuracy: 0.781250
Epoch  1, Batch 245 -Loss:  1587.9390 Validation Accuracy: 0.789062
Epoch  1, Batch 246 -Loss:  2850.1914 Validation Accuracy: 0.789062
Epoch  1, Batch 247 -Loss:  1911.5992 Validation Accuracy: 0.785156
Epoch  1, Batch 248 -Loss:  1306.1665 Validation Accuracy: 0.781250
Epoch  1, Batch 249 -Loss:  1724.9938 Validation Accuracy: 0.777344
Epoch  1, Batch 250 -Loss:  1662.9587 Validation Accuracy: 0.781250
Epoch  1, Batch 251 -Loss:  2314.4207 Validation Accuracy: 0.792969
Epoch  1, Batch 252 -Loss:  1449.5618 Validation Accuracy: 0.789062
Epoch  1, Batch 253 -Loss:  2729.3179 Validation Accuracy: 0.789062
Epoch  1, Batch 254 -Loss:  2528.6826 Validation Accuracy: 0.785156
Epoch  1, Batch 255 -Loss:  1609.9844 Validation Accuracy: 0.789062
Epoch  1, Batch 256 -Loss:  2430.2095 Validation Accuracy: 0.781250
Epoch  1, Batch 257 -Loss:  1443.4155 Validation

Epoch  1, Batch 364 -Loss:  1445.7771 Validation Accuracy: 0.808594
Epoch  1, Batch 365 -Loss:  1719.2484 Validation Accuracy: 0.808594
Epoch  1, Batch 366 -Loss:  1295.4258 Validation Accuracy: 0.804688
Epoch  1, Batch 367 -Loss:   912.9943 Validation Accuracy: 0.808594
Epoch  1, Batch 368 -Loss:  2081.0513 Validation Accuracy: 0.804688
Epoch  1, Batch 369 -Loss:  2910.4106 Validation Accuracy: 0.816406
Epoch  1, Batch 370 -Loss:  1770.1223 Validation Accuracy: 0.816406
Epoch  1, Batch 371 -Loss:  1213.2747 Validation Accuracy: 0.808594
Epoch  1, Batch 372 -Loss:   760.8279 Validation Accuracy: 0.812500
Epoch  1, Batch 373 -Loss:  2032.1827 Validation Accuracy: 0.812500
Epoch  1, Batch 374 -Loss:  1513.7675 Validation Accuracy: 0.816406
Epoch  1, Batch 375 -Loss:  2446.8989 Validation Accuracy: 0.820312
Epoch  1, Batch 376 -Loss:  2024.8788 Validation Accuracy: 0.820312
Epoch  1, Batch 377 -Loss:  1908.3081 Validation Accuracy: 0.812500
Epoch  1, Batch 378 -Loss:  1314.9618 Validation

Epoch  2, Batch  56 -Loss:  1021.2257 Validation Accuracy: 0.839844
Epoch  2, Batch  57 -Loss:  1652.4871 Validation Accuracy: 0.839844
Epoch  2, Batch  58 -Loss:  1300.4137 Validation Accuracy: 0.839844
Epoch  2, Batch  59 -Loss:  1073.5427 Validation Accuracy: 0.832031
Epoch  2, Batch  60 -Loss:  1777.9990 Validation Accuracy: 0.832031
Epoch  2, Batch  61 -Loss:  1640.6980 Validation Accuracy: 0.832031
Epoch  2, Batch  62 -Loss:   827.6104 Validation Accuracy: 0.828125
Epoch  2, Batch  63 -Loss:  1474.1926 Validation Accuracy: 0.828125
Epoch  2, Batch  64 -Loss:  1464.9260 Validation Accuracy: 0.828125
Epoch  2, Batch  65 -Loss:  1640.4935 Validation Accuracy: 0.832031
Epoch  2, Batch  66 -Loss:  1542.0790 Validation Accuracy: 0.824219
Epoch  2, Batch  67 -Loss:   700.2909 Validation Accuracy: 0.824219
Epoch  2, Batch  68 -Loss:  1500.5519 Validation Accuracy: 0.824219
Epoch  2, Batch  69 -Loss:  1691.4873 Validation Accuracy: 0.824219
Epoch  2, Batch  70 -Loss:  1231.6567 Validation

...

Epoch  9, Batch 320 -Loss:   528.5776 Validation Accuracy: 0.910156
Epoch  9, Batch 321 -Loss:   290.8019 Validation Accuracy: 0.906250
Epoch  9, Batch 322 -Loss:   373.5966 Validation Accuracy: 0.906250
Epoch  9, Batch 323 -Loss:   132.3282 Validation Accuracy: 0.906250
Epoch  9, Batch 324 -Loss:   226.5728 Validation Accuracy: 0.906250
Epoch  9, Batch 325 -Loss:   230.7187 Validation Accuracy: 0.906250
Epoch  9, Batch 326 -Loss:   303.9110 Validation Accuracy: 0.906250
Epoch  9, Batch 327 -Loss:   203.7667 Validation Accuracy: 0.906250
Epoch  9, Batch 328 -Loss:   282.0714 Validation Accuracy: 0.906250
Epoch  9, Batch 329 -Loss:   288.4131 Validation Accuracy: 0.906250
Epoch  9, Batch 330 -Loss:   132.2110 Validation Accuracy: 0.906250
Epoch  9, Batch 331 -Loss:   391.6772 Validation Accuracy: 0.906250
Epoch  9, Batch 332 -Loss:   336.8501 Validation Accuracy: 0.906250
Epoch  9, Batch 333 -Loss:   317.3643 Validation Accuracy: 0.902344
Epoch  9, Batch 334 -Loss:   318.3697 Validation

Epoch 10, Batch  12 -Loss:   317.6652 Validation Accuracy: 0.910156
Epoch 10, Batch  13 -Loss:   313.6840 Validation Accuracy: 0.910156
Epoch 10, Batch  14 -Loss:   243.9287 Validation Accuracy: 0.910156
Epoch 10, Batch  15 -Loss:   349.0618 Validation Accuracy: 0.910156
Epoch 10, Batch  16 -Loss:   242.7365 Validation Accuracy: 0.910156
Epoch 10, Batch  17 -Loss:   182.7233 Validation Accuracy: 0.910156
Epoch 10, Batch  18 -Loss:   305.0798 Validation Accuracy: 0.910156
Epoch 10, Batch  19 -Loss:   204.9912 Validation Accuracy: 0.910156
Epoch 10, Batch  20 -Loss:   167.5667 Validation Accuracy: 0.910156
Epoch 10, Batch  21 -Loss:   429.1175 Validation Accuracy: 0.910156
Epoch 10, Batch  22 -Loss:   327.2597 Validation Accuracy: 0.910156
Epoch 10, Batch  23 -Loss:   354.4505 Validation Accuracy: 0.910156
Epoch 10, Batch  24 -Loss:   438.8568 Validation Accuracy: 0.910156
Epoch 10, Batch  25 -Loss:   285.4556 Validation Accuracy: 0.910156
Epoch 10, Batch  26 -Loss:   173.3556 Validation

Epoch 10, Batch 133 -Loss:   332.3384 Validation Accuracy: 0.910156
Epoch 10, Batch 134 -Loss:   261.2422 Validation Accuracy: 0.914062
Epoch 10, Batch 135 -Loss:   200.1881 Validation Accuracy: 0.914062
Epoch 10, Batch 136 -Loss:   394.4687 Validation Accuracy: 0.910156
Epoch 10, Batch 137 -Loss:   246.3703 Validation Accuracy: 0.910156
Epoch 10, Batch 138 -Loss:   360.9169 Validation Accuracy: 0.910156
Epoch 10, Batch 139 -Loss:   222.9879 Validation Accuracy: 0.910156
Epoch 10, Batch 140 -Loss:   265.7240 Validation Accuracy: 0.906250
Epoch 10, Batch 141 -Loss:   140.4942 Validation Accuracy: 0.906250
Epoch 10, Batch 142 -Loss:   195.6301 Validation Accuracy: 0.906250
Epoch 10, Batch 143 -Loss:   148.1078 Validation Accuracy: 0.906250
Epoch 10, Batch 144 -Loss:   360.8687 Validation Accuracy: 0.906250
Epoch 10, Batch 145 -Loss:   293.8074 Validation Accuracy: 0.906250
Epoch 10, Batch 146 -Loss:    36.8431 Validation Accuracy: 0.906250
Epoch 10, Batch 147 -Loss:   188.3073 Validation

Epoch 10, Batch 254 -Loss:   315.2927 Validation Accuracy: 0.906250
Epoch 10, Batch 255 -Loss:   218.1715 Validation Accuracy: 0.906250
Epoch 10, Batch 256 -Loss:   271.1503 Validation Accuracy: 0.910156
Epoch 10, Batch 257 -Loss:   337.8484 Validation Accuracy: 0.906250
Epoch 10, Batch 258 -Loss:   304.0359 Validation Accuracy: 0.906250
Epoch 10, Batch 259 -Loss:   215.9496 Validation Accuracy: 0.906250
Epoch 10, Batch 260 -Loss:   246.7767 Validation Accuracy: 0.902344
Epoch 10, Batch 261 -Loss:   394.8118 Validation Accuracy: 0.902344
Epoch 10, Batch 262 -Loss:   220.6538 Validation Accuracy: 0.906250
Epoch 10, Batch 263 -Loss:   360.6009 Validation Accuracy: 0.910156
Epoch 10, Batch 264 -Loss:   334.9961 Validation Accuracy: 0.910156
Epoch 10, Batch 265 -Loss:   228.1440 Validation Accuracy: 0.910156
Epoch 10, Batch 266 -Loss:   365.5017 Validation Accuracy: 0.914062
Epoch 10, Batch 267 -Loss:   419.3690 Validation Accuracy: 0.914062
Epoch 10, Batch 268 -Loss:   175.8225 Validation

Epoch 10, Batch 375 -Loss:    76.2745 Validation Accuracy: 0.910156
Epoch 10, Batch 376 -Loss:   506.6335 Validation Accuracy: 0.910156
Epoch 10, Batch 377 -Loss:   218.2362 Validation Accuracy: 0.906250
Epoch 10, Batch 378 -Loss:   364.5300 Validation Accuracy: 0.906250
Epoch 10, Batch 379 -Loss:   311.4841 Validation Accuracy: 0.906250
Epoch 10, Batch 380 -Loss:   297.4553 Validation Accuracy: 0.906250
Epoch 10, Batch 381 -Loss:   279.1835 Validation Accuracy: 0.910156
Epoch 10, Batch 382 -Loss:   278.5000 Validation Accuracy: 0.914062
Epoch 10, Batch 383 -Loss:   181.8347 Validation Accuracy: 0.910156
Epoch 10, Batch 384 -Loss:   174.1229 Validation Accuracy: 0.910156
Epoch 10, Batch 385 -Loss:   274.0927 Validation Accuracy: 0.910156
Epoch 10, Batch 386 -Loss:   341.0869 Validation Accuracy: 0.906250
Epoch 10, Batch 387 -Loss:   336.0002 Validation Accuracy: 0.906250
Epoch 10, Batch 388 -Loss:   309.3033 Validation Accuracy: 0.910156
Epoch 10, Batch 389 -Loss:   243.2317 Validation